In [41]:
import random
from itertools import product

# ranks:
# 1 -> Ace
# 2-10 -> 2-10
# 11 -> J
# 12 -> Q
# 13 -> K

class Card:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit
        if self.rank == 1:
            self.card_scores = [1, 11]
        elif self.rank >= 11 and self.rank <= 13:
            self.card_scores = [10, 10]
        else:
            self.card_scores = [self.rank, self.rank]

        if self.rank == 1:
            self.short_rank = 'A'
        elif self.rank == 11:
            self.short_rank = 'J'
        elif self.rank == 12:
            self.short_rank = 'Q'
        elif self.rank == 13:
            self.short_rank = 'K'
        else:
            self.short_rank = str(self.rank)

        if self.suit == 'Spades':
            self.short_suit = 'S'
        elif self.suit == 'Hearts':
            self.short_suit = 'H'
        elif self.suit == 'Clubs':
            self.short_suit = 'C'
        else:
            self.short_suit = 'D'

        self.image_location = 'static/images/{}{}.png'.format(
            self.short_rank, self.short_suit)

    def __repr__(self):
        if self.rank == 1:
            true_rank = 'Ace'
        elif self.rank == 11:
            true_rank = 'Jack'
        elif self.rank == 12:
            true_rank = 'Queen'
        elif self.rank == 13:
            true_rank = 'King'
        else:
            true_rank = str(self.rank)
        return '{} of {}'.format(true_rank, self.suit)

suits = ('Spades', 'Hearts', 'Clubs', 'Diamonds')

class Deck:
    def __init__(self, number_of_decks):
        self.number_of_decks = number_of_decks
        self.cards = []
        self.create(self.number_of_decks)

    def __repr__(self):
        return 'Game deck has {} cards remaining'.format(len(self.cards))

    def create(self, number_of_decks):
        decks = [Card(rank, suit) for suit in suits for rank in range(1, 14)
                 for deck in range(number_of_decks)]
        decks = random.sample(decks, len(decks))
        self.cards.extend(decks)

    def create_test_deck(self,cards):
        self.cards = []
        for card in cards:
            self.cards.append(card)
    
    def draw(self):
        drawn_card = self.cards[0]
        self.cards.remove(self.cards[0])
        return drawn_card

    def reset(self):
        self.cards = []
        self.create(self.number_of_decks)

In [2]:
class Rules:
    def __init__(self):
        self.dealersoft17 = 'stand' #instert 'stand' or 'hit' for dealer on soft 17
        self.blackjack = 1.5 #insert multiplier in case of Blackjack on initial hand
        self.doubledown = [9,10,11] #insert ranks of cards that can be doubled
        self.doublesoft = False
        self.doubleaftersplit = True #insert if Player can double after split
        self.splitaces = True #insert if Player can split Aces
        self.splitnumber = 3 #insert how many parallel sets Player can have
        self.peek = True
rules = Rules()

In [76]:
class Hand:
    def __init__(self,rules,bet):
        self.hand_cards = []
        self.score = [0,0]
        self.options = []
        self.rules = rules
        self.hand_bet = bet
        
    def calculate_score(self):
        # loop over all cards in hand
        self.score = [0,0]
        all_scores = []
        for card in self.hand_cards:
            all_scores.append(card.card_scores)
            
        self.score = sorted(list({sum(combination) for 
                                  combination in 
                                  product(*all_scores)}))[0:2]
        if len(self.score) < 2:
            self.score.append(self.score[0])
            
        if self.score[0] == self.score[1]:
            print('Current Hand score: {0}'.format(self.score[0]))
        elif self.score[1] > 21:
            print('Current Hand score: {0}'.format(self.score[0]))
        else:
            print('Current Hand score: {0} or {1}'.format(self.score[0], self.score[1]))

    def calculate_options(self):
        self.options = []
        if len(self.hand_cards) == 2: #initial possibilities after first draw
            if self.score[1] == 21:
                return
            if self.rules.doublesoft:
                if any(x in self.score for x in self.rules.doubledown):
                    self.options.append('doubledown')
            else:
                if self.score[1] in self.rules.doubledown:
                    self.options.append('doubledown')
            if self.hand_cards[0].card_scores[0] == self.hand_cards[1].card_scores[0]:
                self.options.append('split')
            if self.score[0] <= 21:
                self.options.append('hit')
            self.options.append('stand')
        else: #after other actions have been taken
            if self.score[0] > 21:
                self.options = ['BUST']
            if self.score[0] <= 21:
                self.options.append('hit')
                self.options.append('stand')
        print(self.options)

    def calculate_hand_bet(self):
        print(self.hand_bet)
        
    def show_hand_cards(self):
        print(self.hand_cards)

    def hit(self,deck,num_cards):
        for i in range(0,num_cards):
            self.hand_cards.append(deck.draw())
        self.calculate_score()
        self.show_hand_cards()
        self.calculate_options()
        self.calculate_hand_bet()
        
    def stand(self):
        self.calculate_score()
        self.show_hand_cards()
        self.options = ['none']
        self.calculate_hand_bet()

    def doubledown(self,deck):
        self.hand_bet = 2*self.hand_bet
        self.hand_cards.append(deck.draw())
        self.calculate_score()
        if self.score[0] > 21:
            self.options = ['BUST']
        else:
            self.options = ['none']
        self.show_hand_cards()
        print(self.options)
        self.calculate_hand_bet()

class Player:
    def __init__(self,rules):
        self.funding = 0
        self.hands = []
        self.score = [0,0]
        self.player_options = []
        self.rules = rules

    def refund(self,funds):
        self.funding = funds
    
    def initiate(self, deck, bet = 0):
        self.bet = bet
        self.reset_hands()
        self.add_hand()
        self.hands[0].hit(deck,2)

        if any(x in self.hands[0].score for x in [21]): #in case of blackjack no more options can be done
            self.options = ['blackjack']
            print('blackjack on initial')
    
    def add_hand(self):
        hand = Hand(self.rules, self.bet)
        self.hands.append(hand)

    def show_options(self, hand_index):
        print(self.hands[hand_index].options)
    
    def reset_hands(self):
        self.hands = []

    def hit(self, deck, hand_index):
        self.hands[hand_index].hit(deck,1)

    def stand(self, hand_index):
        self.hands[hand_index].stand()

    def doubledown(self, deck, hand_index):
        self.hands[hand_index].doubledown(deck)

    def split(self, deck, hand_index):
        hand = Hand(self.rules, self.bet)
        hand.hand_cards.append(self.hands[hand_index].hand_cards[1])
        self.hands[hand_index].hand_cards.pop()
        
        self.hands.append(hand)

        for i in [hand_index,len(self.hands)-1]:
            self.hands[i].hit(deck,1)

class Dealer:
    def __init__(self,rules):
        self.hand_cards = []
        self.score = [0,0]
        self.options = []
        self.rules = rules
        
    def calculate_score(self, initial = False):
        # loop over all cards in hand
       
        if initial:
            self.score = [0,0]
            self.score[0] += self.hand_cards[0].card_scores[0]
            self.score[1] += self.hand_cards[0].card_scores[1]
        else:
            self.score = [0,0]
            all_scores = []
            for card in self.hand_cards:
                all_scores.append(card.card_scores)
                
            self.score = sorted(list({sum(combination) for 
                                      combination in 
                                      product(*all_scores)}))[0:2]
            if len(self.score) < 2:
                self.score.append(self.score[0])
        
        if self.score[0] == self.score[1]:
            print('Current Hand score: {0}'.format(self.score[0]))
        elif self.score[1] > 21:
            print('Current Hand score: {0}'.format(self.score[0]))
        else:
            print('Current Hand score: {0} or {1}'.format(self.score[0], self.score[1]))     
    
    def initiate(self, deck):
        self.reset_hand()
        self.hand_cards.append(deck.draw())
        self.hand_cards.append(deck.draw())

        self.calculate_score(initial = True)
        if self.hand_cards[0].rank == 1:
            self.options = ['peek']
        else:
            self.options = ['reveal']
        print(self.options)
        
    def reveal(self):
        self.calculate_score()
        self.calculate_options()
    
    def hit(self, deck):
        self.hand_cards.append(deck.draw())

        self.calculate_score()
        self.calculate_options()

    def peek(self):
        if self.hand_cards[1].rank in [10,11,12,13]:
            self.options = ['blackjack']
        else:
            self.options = ['reveal']
        print(self.options)
    
    def calculate_options(self):
        self.options = []
        if self.score[0] == self.score[1]:
            if self.score[0] < 17:
                self.options.append('hit')
            elif 17 <= self.score[0] < 21:
                self.options.append('none')
            else:
                self.options.append('BUST')
        else:
            if 17 <= self.score[1] < 21:
                self.options.append('none')
            elif self.score[0] > 21:
                self.options.append('BUST')
            else:
                self.options.append('hit')
        print(self.options)
        
    def reset_hand(self):
        self.hand_cards = []

In [77]:
test_deck = Deck(1)
test_deck.create_test_deck([Card(1,'Hearts'),
                                       Card(1,'Spades'),
                           Card(2,'Hearts'),
                           Card(3,'Diamonds')])

In [86]:
game_deck = Deck(1)

In [87]:
player = Player(rules)
dealer = Dealer(rules)

In [88]:
game_deck.__repr__()

'Game deck has 52 cards remaining'

In [89]:
player.initiate(game_deck)

Current Hand score: 11 or 21
[Ace of Hearts, 10 of Hearts]
0
blackjack on initial


In [90]:
dealer.initiate(game_deck)

Current Hand score: 4
['reveal']


In [91]:
game_deck.__repr__()

'Game deck has 48 cards remaining'

In [92]:
dealer.reveal()

Current Hand score: 14
['hit']


In [93]:
game_deck.__repr__()

'Game deck has 48 cards remaining'

In [94]:
dealer.hit(game_deck)

Current Hand score: 17
['none']


In [95]:
game_deck.__repr__()

'Game deck has 47 cards remaining'

## Testing

In [31]:
deck = Deck(8)

def simulate_draws(num_decks):
    """Simulate drawing cards from multiple decks."""
    results = []
    for _ in range(52 * num_decks):  # Multiply by number of decks
        results.append(str(deck.draw()))
    return results

def count_cards(cards):
    """Count occurrences of each card in the list."""
    counts = {}
    for card_instance in cards:
        card = str(card_instance)
        if card in counts:
            counts[card] += 1
        else:
            counts[card] = 1
    return counts
results = simulate_draws(deck.number_of_decks)
# Use the function to count cards
card_counts = count_cards(results)

# Optionally, print the total number of unique cards
print(f"Total number of unique cards: {len(card_counts)}")

# Print each card and its count
for card, count in card_counts.items():
    print(f"{card}: {count}")

Total number of unique cards: 52
8 of Clubs: 8
8 of Hearts: 8
5 of Spades: 8
9 of Clubs: 8
7 of Hearts: 8
Jack of Diamonds: 8
7 of Clubs: 8
Ace of Clubs: 8
9 of Spades: 8
4 of Diamonds: 8
2 of Hearts: 8
6 of Spades: 8
Ace of Hearts: 8
10 of Spades: 8
Queen of Hearts: 8
King of Hearts: 8
4 of Hearts: 8
2 of Diamonds: 8
7 of Diamonds: 8
2 of Spades: 8
3 of Diamonds: 8
Queen of Diamonds: 8
King of Diamonds: 8
5 of Hearts: 8
King of Clubs: 8
10 of Diamonds: 8
3 of Clubs: 8
6 of Diamonds: 8
8 of Diamonds: 8
10 of Hearts: 8
3 of Hearts: 8
9 of Hearts: 8
3 of Spades: 8
4 of Clubs: 8
Queen of Clubs: 8
2 of Clubs: 8
5 of Clubs: 8
4 of Spades: 8
Jack of Spades: 8
6 of Hearts: 8
Queen of Spades: 8
5 of Diamonds: 8
8 of Spades: 8
King of Spades: 8
Jack of Clubs: 8
9 of Diamonds: 8
Ace of Diamonds: 8
Ace of Spades: 8
10 of Clubs: 8
Jack of Hearts: 8
6 of Clubs: 8
7 of Spades: 8


## Cheatsheet

In [ ]:
class Player(Dealer):
    def __init__(self):
        self.cards = []
        self.hand_scores = [0, 0]
        self.best_outcome = 'Awaiting deal'
        self.possible_actions = ['No deal yet']

    def __repr__(self):
        return 'Player Hand: {}, Scores: {}, Best Outcome: {}'.format(self.cards, list(set(self.hand_scores)), self.best_outcome)

    def stand(self, game_play):
        self.possible_actions = []
        game_play.commentary.append('Player is standing')

    def double_down(self, game_deck, game_play):
        self.hit(game_deck)
        game_play.commentary.append('Player is doubling down')
        self.possible_actions = []

    def player_hit(self, game_deck, game_play):
        self.hit(game_deck)
        game_play.commentary.append('Player has hit')
        self.get_possibilities(game_play)

    def get_possibilities(self, game_play):
        if self.best_outcome in ['Blackjack', 'Bust', 21]:
            self.possible_actions = []
            game_play.commentary.append('Player has no options')
        elif len(self.cards) == 2:
            self.possible_actions = ['Hit', 'Stand', 'Double Down']
            game_play.commentary.append(
                'Player can still hit, double down or stand')
        else:
            self.possible_actions = ['Hit', 'Stand']
            game_play.commentary.append('Player can still hit or stand')

    def reset(self):
        self.cards = []
        self.hand_scores = [0, 0]
        self.best_outcome = 'Awaiting deal'
        self.possible_actions = []
        self.has_doubled_down = False